# Custom Training Loops in Keras

In [1]:
import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


## Define The Model

In [2]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])


## Define Loss Function and Optimizer

In [3]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

## Implement the Custom Training Loop

In [ ]:
epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Update weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.327298641204834
Epoch 1 Step 200: Loss = 0.36879265308380127
Epoch 1 Step 400: Loss = 0.20166833698749542
Epoch 1 Step 600: Loss = 0.12539997696876526
Epoch 1 Step 800: Loss = 0.13895612955093384
Epoch 1 Step 1000: Loss = 0.444985568523407
Epoch 1 Step 1200: Loss = 0.17133165895938873
Epoch 1 Step 1400: Loss = 0.2113090455532074
Epoch 1 Step 1600: Loss = 0.21189343929290771
Epoch 1 Step 1800: Loss = 0.19413301348686218
Start of epoch 2
Epoch 2 Step 0: Loss = 0.09056811034679413
Epoch 2 Step 200: Loss = 0.14047843217849731
Epoch 2 Step 400: Loss = 0.11003763228654861
Epoch 2 Step 600: Loss = 0.048787884414196014
Epoch 2 Step 800: Loss = 0.0851454883813858
Epoch 2 Step 1000: Loss = 0.267480731010437
Epoch 2 Step 1200: Loss = 0.10105648636817932
Epoch 2 Step 1400: Loss = 0.12533798813819885
Epoch 2 Step 1600: Loss = 0.14928655326366425
Epoch 2 Step 1800: Loss = 0.1257006973028183


## Define Loss Function and Optimizer (Add accuracy metric)

In [5]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  
optimizer = tf.keras.optimizers.Adam()
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

## Implement the custom training loop with accuracy

In [7]:
epochs = 5

for epoch in range(epochs):
    print(f'Start of eopch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass : Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
            
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)
        
        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}, Accuracy = {accuracy_metric.result().numpy()}')
            
    # Reset accuracy metric at the end of each epoch
    accuracy_metric.reset_state()

Start of eopch 1
Epoch 1 Step 0: Loss = 0.018028568476438522, Accuracy = 0.9752465486526489
Epoch 1 Step 200: Loss = 0.07714109122753143, Accuracy = 0.9760657548904419
Epoch 1 Step 400: Loss = 0.06679058074951172, Accuracy = 0.9763702750205994
Epoch 1 Step 600: Loss = 0.044337447732686996, Accuracy = 0.9768780469894409
Epoch 1 Step 800: Loss = 0.029298681765794754, Accuracy = 0.9773098826408386
Epoch 1 Step 1000: Loss = 0.1106564849615097, Accuracy = 0.9778012037277222
Epoch 1 Step 1200: Loss = 0.04148627072572708, Accuracy = 0.9780965447425842
Epoch 1 Step 1400: Loss = 0.03512387350201607, Accuracy = 0.9784131050109863
Epoch 1 Step 1600: Loss = 0.05119360610842705, Accuracy = 0.978621244430542
Epoch 1 Step 1800: Loss = 0.04453087970614433, Accuracy = 0.9789513349533081
Start of eopch 2
Epoch 2 Step 0: Loss = 0.014405915513634682, Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.06961792707443237, Accuracy = 0.9875621795654297
Epoch 2 Step 400: Loss = 0.03687578812241554, Accuracy = 0.9872194